In [1]:
import numpy as np
import 
from jupyterHelpers import *

thisYr = "2021"
rnds = ["15"]

print("done")

done


In [2]:
# updateSes(thisYr, rnds)
print("done")


done


In [4]:
###  UNACCEPTABLE
frame = getWholeFrame()

frame.()

,index,month,day,yr,lge,rnd,session,trk,rdr_num,f_name,...,four_seconds,avg_spd,start,lap_scaled,one_scaled,two_scaled,thr_scaled,four_scaled,avgSpd_scaled,results
0,2005-125cc-1-FP1-1000,April,8,2005,125cc,1,FP1,Circuito de Jerez,6.0,joan,...,38.490,NaN,NaN,0.183740,0.339167,0.517903,0.793179,0.065690,NaN,0-pos
1,2005-125cc-1-FP1-1001,April,8,2005,125cc,1,FP1,Circuito de Jerez,6.0,joan,...,36.938,NaN,NaN,0.122286,0.121593,0.470725,0.737287,0.063041,NaN,0-pos
2,2005-125cc-1-FP1-1002,April,8,2005,125cc,1,FP1,Circuito de Jerez,6.0,joan,...,36.226,NaN,NaN,0.118395,0.115732,0.457738,0.712849,0.061826,NaN,0-pos
3,2005-125cc-1-FP1-1003,April,8,2005,125cc,1,FP1,Circuito de Jerez,6.0,joan,...,35.525,NaN,NaN,0.116406,0.112753,0.454167,0.704855,0.060630,NaN,0-pos
4,2005-125cc-1-FP1-1004,April,8,2005,125cc,1,FP1,Circuito de Jerez,6.0,joan,...,35.988,NaN,NaN,0.116323,0.111692,0.448298,0.704008,0.061420,NaN,0-pos


In [5]:
frame.tail()

,index,month,day,yr,lge,rnd,session,trk,rdr_num,f_name,...,four_seconds,avg_spd,start,lap_scaled,one_scaled,two_scaled,thr_scaled,four_scaled,avgSpd_scaled,results
2439,2021-GP-9-WUP-1216,June,27,2021,MotoGP,9,WUP,TT Circuit Assen,31.0,garrett,...,21.917,300.7,22.0,0.379494,0.896269,0.759658,0.759454,0.650453,0.955817,17th
2440,2021-GP-9-WUP-1217,June,27,2021,MotoGP,9,WUP,TT Circuit Assen,31.0,garrett,...,28.401,301.4,22.0,0.366765,0.682029,0.745808,0.757490,0.842885,0.958042,17th
2441,2021-GP-9-WUP-1218,June,27,2021,MotoGP,9,WUP,TT Circuit Assen,31.0,garrett,...,22.031,301.4,22.0,0.344663,0.686451,0.749204,0.758846,0.653836,0.958042,17th
2442,2021-GP-9-WUP-1219,June,27,2021,MotoGP,9,WUP,TT Circuit Assen,31.0,garrett,...,21.837,301.4,22.0,0.343437,0.683696,0.742942,0.761612,0.648078,0.958042,17th
2443,2021-GP-9-WUP-1220,June,27,2021,MotoGP,9,WUP,TT Circuit Assen,31.0,garrett,...,21.869,301.4,22.0,0.345017,0.683807,0.755997,0.765762,0.649028,0.958042,17th


In [6]:
# pandas join function
# gradient boosted trees
# xgboost
# neural net
